In [36]:
import pandas as pd
import numpy as np

In [37]:
"""
Nominal rate is total annual net tax billing expressed as percentage of net
taxable assessed value. Effective rate is total annual net tax billing expressed
as percentage of sales price. Assessment ratio is the assessment-sales price ratio(assessed value/sales price).
Except for single-family homes, all rates and ratios are medians based on
samples of measurable sales. All rates and ratios are percentages(1 means 1%).

More details about the spreadsheet can be found in "Full TPV Historical Database.xls"/CoArea4_Footnotes
"""

str_columns = ['ID', 'State', 'Type', 'County', 'Area', 'Year']
data = pd.read_excel(
    "../../../data/raw/Prop 13 Resources/Prop-13 Impact Analysis (Kendall)/Tax Rates/TPV - Table CoArea4. Median Property Tax Rates and Assessment Ratios for Ordinary Real Property Involved in Measurable Sales.xlsx",
    usecols="A:E,H,K,M,W,Y,AI",
    header=0,
    skiprows=[i for i in range(11)],
    dtype={col: str for col in str_columns}
)
data = data.rename(columns={
    'Nom_Tax_Rate_Res_Tot': 'Nom_Tax_Rate_All_Res_Med',
    'Nom_Tax_Rate_Res_Sgl_All_Med': 'Nom_Tax_Rate_Res_All_SFH_Med',
    'Eff_Tax_Rate_Res_Tot': 'Eff_Tax_Rate_All_Res_Med',
    'Eff_Tax_Rate_Res_Sgl_All_Med': 'Eff_Tax_Rate_Res_All_SFH_Med',
    'AV_Ratio_Res_Tot': 'AV_Ratio_All_Res_Med'
})
VAL_COLUMNS = ['Nom_Tax_Rate_All_Res_Med', 'Nom_Tax_Rate_Res_All_SFH_Med',
               'Eff_Tax_Rate_All_Res_Med', 'Eff_Tax_Rate_Res_All_SFH_Med',
               'AV_Ratio_All_Res_Med']

In [38]:
GOVS_TYPE_MAP = {"1": "County", "2": "City", "9": "Balance of County"}

In [39]:
data.columns

Index(['ID', 'State', 'Type', 'County', 'Area', 'Year',
       'Nom_Tax_Rate_All_Res_Med', 'Nom_Tax_Rate_Res_All_SFH_Med',
       'Eff_Tax_Rate_All_Res_Med', 'Eff_Tax_Rate_Res_All_SFH_Med',
       'AV_Ratio_All_Res_Med'],
      dtype='object')

In [40]:
data.head()

,ID,State,Type,County,Area,Year,Nom_Tax_Rate_All_Res_Med,Nom_Tax_Rate_Res_All_SFH_Med,Eff_Tax_Rate_All_Res_Med,Eff_Tax_Rate_Res_All_SFH_Med,AV_Ratio_All_Res_Med
0,01037,01,1,037,"Jefferson County, AL",1982,5.26,5.19,0.33,0.31,6.24
1,012037003,01,2,037,Birmingham city,1982,6.08,6.01,0.37,0.35,6.62
2,019037999,01,9,037,Balance of county,1982,4.21,4.21,0.26,0.26,6.03
3,01049,01,1,049,"Mobile County, AL",1982,6.03,5.89,0.14,0.14,5.89
4,012049004,01,2,049,Mobile city,1982,6.63,6.63,0.14,0.14,5.20


In [41]:
data.shape

(2209, 11)

In [42]:
data.dtypes

ID                               object
State                            object
Type                             object
County                           object
Area                             object
Year                             object
Nom_Tax_Rate_All_Res_Med        float64
Nom_Tax_Rate_Res_All_SFH_Med    float64
Eff_Tax_Rate_All_Res_Med        float64
Eff_Tax_Rate_Res_All_SFH_Med    float64
AV_Ratio_All_Res_Med            float64
dtype: object

In [43]:
data.describe()

,Nom_Tax_Rate_All_Res_Med,Nom_Tax_Rate_Res_All_SFH_Med,Eff_Tax_Rate_All_Res_Med,Eff_Tax_Rate_Res_All_SFH_Med,AV_Ratio_All_Res_Med
count,2209.000000,2209.000000,2209.000000,2209.000000,2209.000000
mean,-4768.653033,-3978.388094,-4606.304097,-3816.485921,-4551.650883
std,6093.583704,6050.706194,6067.067260,6002.370969,6048.484124
min,-22222.000000,-22222.000000,-22222.000000,-22222.000000,-22222.000000
25%,-11111.000000,-11111.000000,-11111.000000,-11111.000000,-11111.000000
50%,2.700000,4.000000,0.950000,1.170000,16.000000
75%,7.870000,8.460000,1.810000,1.940000,30.600000
max,63.030000,63.030000,7.120000,6.770000,130.200000


In [44]:
# clean str columns
for col in str_columns:
    data[col] = data[col].str.strip()
    data[col] = data[col].str.lower()
    data[col] = data[col].map(lambda x: None if x == "" else x)

In [45]:
# check number of missing values for each col
def check_missing_values_all_cols(df):
    col_names = df.columns
    missing_val_dict = dict()
    for col in col_names:
        missing_cnt = sum(df[col].isna())
        if missing_cnt > 0:
            missing_val_dict[col] = missing_cnt
    print(missing_val_dict)

check_missing_values_all_cols(data)

{'Year': 1}


In [46]:
data['Year'].unique()

array(['1982', '1977', nan, '1972', '1967'], dtype=object)

In [47]:
# maybe 1972, needs to investigate
data[data['Year'].isna()]

,ID,State,Type,County,Area,Year,Nom_Tax_Rate_All_Res_Med,Nom_Tax_Rate_Res_All_SFH_Med,Eff_Tax_Rate_All_Res_Med,Eff_Tax_Rate_Res_All_SFH_Med,AV_Ratio_All_Res_Med
884,393023020,39,3,023,upper darby township,NaN,23.67,23.67,1.8,1.8,7.6


In [48]:
# remove rows with missing year
data = data[~data['Year'].isna()].copy()
print(data.shape)

(2208, 11)


In [49]:
'''-11111 = Not available.
   -22222 = Data not presented where sample representation insufficient, cost'''
for col in VAL_COLUMNS:
    data.loc[(data[col] == -11111) | (data[col] == -22222), col] = None

In [50]:
data.describe()

,Nom_Tax_Rate_All_Res_Med,Nom_Tax_Rate_Res_All_SFH_Med,Eff_Tax_Rate_All_Res_Med,Eff_Tax_Rate_Res_All_SFH_Med,AV_Ratio_All_Res_Med
count,1320.000000,1489.000000,1353.000000,1522.000000,1357.000000
mean,7.849076,7.809960,1.735735,1.727858,33.378482
std,5.900907,5.798709,0.923117,0.872230,21.880098
min,0.720000,0.720000,0.100000,0.000000,1.380000
25%,3.795000,3.890000,1.100000,1.100000,17.740000
50%,6.820000,6.800000,1.600000,1.600000,26.500000
75%,10.637500,10.500000,2.200000,2.200000,43.950000
max,63.030000,63.030000,7.120000,6.770000,130.200000


In [51]:
check_missing_values_all_cols(data)

{'Nom_Tax_Rate_All_Res_Med': 888, 'Nom_Tax_Rate_Res_All_SFH_Med': 719, 'Eff_Tax_Rate_All_Res_Med': 855, 'Eff_Tax_Rate_Res_All_SFH_Med': 686, 'AV_Ratio_All_Res_Med': 851}


In [52]:
# check if ID + Year is unique for each row
print((data['ID'] + '_' + data['Year']).unique().size)

2167


In [53]:
pd.set_option('display.max_rows', None)
# cities/BOC share GOVS ID, but County ID is different
dups = data.groupby(['ID', 'Year']).size().sort_values(ascending=False)
dups = dups[dups > 1]
dups = dups.rename("dup_cnt")
dup_data = data[['ID', 'State', 'Type', 'County', 'Area', 'Year']].merge(dups, how="right", left_on=['ID', 'Year'], right_index=True)
# check if dups are within CA
sum(dup_data["State"] == "05")

0

In [54]:
# GOVS State Code can be found in "Full TPV Historical Database.xls"/CoArea4_Footnotes
CA_data = data[data["State"] == "05"].copy()
CA_data['Type'] = CA_data['Type'].map(lambda x: GOVS_TYPE_MAP[x])
assert CA_data.groupby(["ID", "Year"]).size().shape[0] == CA_data.shape[0]

In [55]:
# correct typo
print(CA_data[CA_data["ID"]=="052030504"])

             ID State  Type County               Area  Year  \
55    052030504    05  City    030  westminister city  1982   
1077  052030504    05  City    030   westminster city  1972   

      Nom_Tax_Rate_All_Res_Med  Nom_Tax_Rate_Res_All_SFH_Med  \
55                        1.22                          1.22   
1077                     12.50                         12.50   

      Eff_Tax_Rate_All_Res_Med  Eff_Tax_Rate_Res_All_SFH_Med  \
55                        0.59                          0.59   
1077                      2.00                          2.00   

      AV_Ratio_All_Res_Med  
55                   53.18  
1077                 19.30  


In [56]:
CA_data.loc[55, "Area"] = "westminster city" 
assert CA_data.groupby("ID").size().shape[0] == CA_data.groupby(["ID", "Area"]).size().shape[0]
CA_DATA_ID2AREA = CA_data.groupby(["ID", "State", "County", "Area"]).size().reset_index()[["ID", "State", "County", "Area"]]
assert CA_DATA_ID2AREA.shape[0] == CA_data.groupby("ID").size().shape[0]

In [57]:
CA_DATA_ID2AREA

,ID,State,County,Area
0,05001,05,001,"alameda county, ca"
1,05007,05,007,"contra costa county, ca"
2,05010,05,010,"fresno county, ca"
3,05015,05,015,"kern county, ca"
4,05019,05,019,"los angeles county, ca"
5,05027,05,027,"monterey county, ca"
6,05030,05,030,"orange county, ca"
7,05033,05,033,"riverside county, ca"
8,05034,05,034,"sacramento county, ca"
9,05036,05,036,"san bernardino county, ca"


In [58]:
# correct wrong GOV Type
assert CA_data[(CA_data["Type"] == "County") & (CA_data["ID"].str.len() != 5)].shape[0] == 0
assert CA_data[(CA_data["Type"] == "City") & (CA_data["ID"].str.len() != 9)].shape[0] == 0
assert CA_data[(CA_data["Type"] == "City") & (CA_data["ID"].str[-3::] == "999")].shape[0] == 0

CA_data[(CA_data["Type"] == "Balance of County") & (CA_data["ID"].str[-3::] != "999")]
CA_data.loc[CA_data["ID"] == "052037009", "Type"] = "City" #oceanside city
assert CA_data[(CA_data["Type"] == "Balance of County") & (CA_data["ID"].str[-3::] != "999")].shape[0] == 0

In [59]:
# Descriptive statestics by year
# all geographic areas
stat_by_year = CA_data[["Year"] + VAL_COLUMNS].groupby(["Year"]).agg([min, np.median, max])
stat_by_year
# No data for Nom_Tax_Rate_All_Res_Med, Eff_Tax_Rate_All_Res_Med, AV_Ratio_All_Res_Med in 1967

Nom_Tax_Rate_All_Res_Med                Nom_Tax_Rate_Res_All_SFH_Med  \
                          min  median    max                          min   
Year                                                                        
1967                      NaN     NaN    NaN                          7.6   
1972                      9.1  12.200  18.10                          9.1   
1977                      9.1  12.060  20.17                          9.1   
1982                      1.0   1.165   1.42                          1.0   

                   Eff_Tax_Rate_All_Res_Med               \
     median    max                      min median   max   
Year                                                       
1967   9.52  13.38                      NaN    NaN   NaN   
1972  12.25  18.10                     1.60   2.30  3.40   
1977  12.06  20.26                     1.16   1.80  2.90   
1982   1.17   1.43                     0.31   0.67  1.13   

     Eff_Tax_Rate_Res_All_SFH_Med              AV_Ratio_All_Res_Med         \
                              min median   max                  min median   
Year                                                                         
1967                         0.88   1.82  2.24                  NaN    NaN   
1972                         1.60   2.20  3.30                16.10  20.30   
1977                         1.12   1.76  2.90                13.00  18.50   
1982                         0.30   0.66  0.91                28.11  61.64   

             
        max  
Year         
1967    NaN  
1972  24.70  
1977  20.30  
1982  84.33

In [61]:
# County Level
county_by_year = CA_data.loc[CA_data["Type"] == "County",
                             ["Year"] + VAL_COLUMNS].groupby(["Year"]).agg([min, np.median, max])
county_by_year.to_csv("test.csv", na_rep="NaN")

In [62]:
# City Level
city_by_year = CA_data.loc[CA_data["Type"] == "City", 
                           ["Year"] + VAL_COLUMNS].groupby(["Year"]).agg([min, np.median, max])
city_by_year.to_csv("test.csv", na_rep="NaN")

In [63]:
# BOC Level
BOC_by_year = CA_data.loc[CA_data["Type"] == "Balance of County", 
                          ["Year"] + VAL_COLUMNS].groupby(["Year"]).agg([min, np.median, max])
BOC_by_year.to_csv("test.csv", na_rep="NaN")

In [28]:
# How many geographical entities do have valid data?
na_by_geo_and_year = CA_data[["Type", "Year"] + VAL_COLUMNS].copy()
na_by_geo_and_year = na_by_geo_and_year.groupby(["Type", "Year"]).count()
na_by_geo_and_year

Nom_Tax_Rate_All_Res_Med  \
Type              Year                             
Balance of County 1967                         0   
                  1972                        19   
                  1977                        17   
                  1982                         7   
City              1967                         0   
                  1972                        61   
                  1977                        58   
                  1982                        34   
County            1967                         0   
                  1972                         0   
                  1977                        18   
                  1982                         7   

                        Nom_Tax_Rate_Res_All_SFH_Med  \
Type              Year                                 
Balance of County 1967                             7   
                  1972                            19   
                  1977                            17   
                  1982                             7   
City              1967                            14   
                  1972                            61   
                  1977                            58   
                  1982                            31   
County            1967                             0   
                  1972                             0   
                  1977                            18   
                  1982                             7   

                        Eff_Tax_Rate_All_Res_Med  \
Type              Year                             
Balance of County 1967                         0   
                  1972                        19   
                  1977                        17   
                  1982                         7   
City              1967                         0   
                  1972                        61   
                  1977                        58   
                  1982                        34   
County            1967                         0   
                  1972                         0   
                  1977                        18   
                  1982                         7   

                        Eff_Tax_Rate_Res_All_SFH_Med  AV_Ratio_All_Res_Med  
Type              Year                                                      
Balance of County 1967                             7                     0  
                  1972                            19                    19  
                  1977                            17                    17  
                  1982                             7                     7  
City              1967                            14                     0  
                  1972                            61                    61  
                  1977                            58                    58  
                  1982                            31                    34  
County            1967                             0                     0  
                  1972                             0                     0  
                  1977                            18                    18  
                  1982                             7                     7

In [29]:
cnt_by_geo = CA_data.groupby("Type").agg({"ID": "nunique"}).rename(columns={"ID": "cnt"})
cnt_by_geo

,cnt
Type,
Balance of County,19
City,72
County,19


In [30]:
year_pairs = ["1977_1982", "1972_1982", "1967_1982"]
gov_types = list(GOVS_TYPE_MAP.values())
val_cols = VAL_COLUMNS + ["All"]
arrays = [
    np.array(year_pairs),
    np.array(val_cols),
]
cnt_tbl = pd.DataFrame([[0] * len(gov_types) for i in range(len(arrays[0]) * len(arrays[1]))],
                       index=pd.MultiIndex.from_product(arrays, names=["years", "values"]), columns=gov_types)
consistent_area_res = {g:{y:{col:0 for col in val_cols} for y in year_pairs} for g in gov_types}

In [31]:
for gov_type in gov_types:
    for year_pair in year_pairs:
        y1, y2 = year_pair.split("_")
        for col in VAL_COLUMNS:
#             print(gov_type, year_pair, col)
            _data = CA_data.loc[CA_data["Type"] == gov_type, ["ID", "Year", col]].copy()
            pivot_data = _data.pivot(index=["ID"], columns="Year", values=col)
            _govs = pivot_data.loc[(~pivot_data[y1].isna()) & (~pivot_data[y2].isna()),:]
            cnt_tbl.loc[(year_pair, col), gov_type] = _govs.shape[0]
            consistent_area_res[gov_type][year_pair][col] = list(_govs.index)
        
        # all indicators are valid
        _data = CA_data.loc[CA_data["Type"] == gov_type, ["ID", "Year"] + VAL_COLUMNS].set_index(["ID", "Year"]).copy()
        _data = _data.isna().sum(axis=1).to_frame().reset_index()
        pivot_data = _data.pivot(index="ID", columns="Year")
        consis_data = pivot_data[(pivot_data.loc[:,(0, y1)] == 0) & (pivot_data.loc[:,(0, y2)] == 0)]
        cnt_tbl.loc[(year_pair, "All"), gov_type] = consis_data.shape[0]
        consistent_area_res[gov_type][year_pair]["All"] = list(consis_data.index)
        

In [32]:
cnt_tbl.T

years                            1977_1982                               \
values            Nom_Tax_Rate_All_Res_Med Nom_Tax_Rate_Res_All_SFH_Med   
County                                   6                            6   
City                                    21                           18   
Balance of County                        6                            6   

years                                                                    \
values            Eff_Tax_Rate_All_Res_Med Eff_Tax_Rate_Res_All_SFH_Med   
County                                   6                            6   
City                                    21                           18   
Balance of County                        6                            6   

years                                                     1972_1982  \
values            AV_Ratio_All_Res_Med All Nom_Tax_Rate_All_Res_Med   
County                               6   6                        0   
City                                21  18                       27   
Balance of County                    6   6                        7   

years                                                                    \
values            Nom_Tax_Rate_Res_All_SFH_Med Eff_Tax_Rate_All_Res_Med   
County                                       0                        0   
City                                        25                       27   
Balance of County                            7                        7   

years                                                                    \
values            Eff_Tax_Rate_Res_All_SFH_Med AV_Ratio_All_Res_Med All   
County                                       0                    0   0   
City                                        25                   27  25   
Balance of County                            7                    7   7   

years                            1967_1982                               \
values            Nom_Tax_Rate_All_Res_Med Nom_Tax_Rate_Res_All_SFH_Med   
County                                   0                            0   
City                                     0                           12   
Balance of County                        0                            7   

years                                                                    \
values            Eff_Tax_Rate_All_Res_Med Eff_Tax_Rate_Res_All_SFH_Med   
County                                   0                            0   
City                                     0                           12   
Balance of County                        0                            7   

years                                       
values            AV_Ratio_All_Res_Med All  
County                               0   0  
City                                 0   0  
Balance of County                    0   0

In [35]:
assert sum(CA_data['Nom_Tax_Rate_Res_All_SFH_Med'].isna() ^ CA_data['Eff_Tax_Rate_Res_All_SFH_Med'].isna()) == 0
# have 1 = have all
assert sum(~((CA_data['Nom_Tax_Rate_All_Res_Med'].isna() & CA_data['Eff_Tax_Rate_All_Res_Med'].isna() & CA_data['AV_Ratio_All_Res_Med'].isna()) | 
   ~(CA_data['Nom_Tax_Rate_All_Res_Med'].isna() | CA_data['Eff_Tax_Rate_All_Res_Med'].isna() | CA_data['AV_Ratio_All_Res_Med'].isna()))) == 0

0

18 cities have all 5 valid indicators for both 1977 and 1982, while 3 cities have valid Nom_Tax_Rate_Res_All_SFH_Med and Eff_Tax_Rate_Res_All_SFH_Med. 6 counties and their resepctive BOC have all 5 valid indicators for both 1977 and 1982.

25 cities have all 5 valid indicators for both 1972 and 1982, while 2 cities have valid Nom_Tax_Rate_Res_All_SFH_Med and Eff_Tax_Rate_Res_All_SFH_Med.
Only 7 BOC have all 5 valid indicators for both 1972 and 1982, and 6 of them are BOCs that we have all valid indicators for both 1972 and 1982. The new BOC is of Orange County. No counties have any valid indicator for both 1972 and 1982, because there is no data for county level at 1972.

12 cities have valid Nom_Tax_Rate_Res_All_SFH_Med and Eff_Tax_Rate_Res_All_SFH_Med for both 1967 and 1982.
The same 7 BOC have valid Nom_Tax_Rate_Res_All_SFH_Med and Eff_Tax_Rate_Res_All_SFH_Med for both 1967 and 1982.

In [34]:
def lookup_for_area_by_ID(list_of_ID):
    _df = pd.DataFrame({"ID": list_of_ID})
    res = _df.merge(CA_DATA_ID2AREA, how="left", on="ID")
    assert _df.shape[0] == res.shape[0]
    return res

In [35]:
lookup_for_area_by_ID(consistent_area_res["Balance of County"]["1977_1982"]["Nom_Tax_Rate_All_Res_Med"])

,ID,State,County,Area
0,059001999,05,001,balance of county
1,059010999,05,010,balance of county
2,059019999,05,019,balance of county
3,059034999,05,034,balance of county
4,059037999,05,037,balance of county
5,059043999,05,043,balance of county


In [36]:
def check_if_boc_align_with_county(boc_df, county_df):
    if boc_df.shape[0] != county_df.shape[0]:
        return False
    
    county_code4boc = boc_df["County"].tolist()
    county_code4county = county_df["County"].tolist()
    
    return ((len(set(county_code4boc)) == len(county_code4boc)) &
            (len(set(county_code4county)) == len(county_code4county)) &
            (set(county_code4boc) == set(county_code4county)))
    
    

In [37]:
for year_pair in year_pairs:
    for col in VAL_COLUMNS:
        boc_df = lookup_for_area_by_ID(consistent_area_res["Balance of County"][year_pair][col])
        county_df = lookup_for_area_by_ID(consistent_area_res["County"][year_pair][col])
        if not check_if_boc_align_with_county(boc_df, county_df):
            print(year_pair, col)
            print("BOC:", boc_df)
            print("County:", county_df)

"""Print out because No data at county level for 1972. No data at county level for 1967."""

1972_1982 Nom_Tax_Rate_All_Res_Med
BOC:           ID State County               Area
0  059001999    05    001  balance of county
1  059010999    05    010  balance of county
2  059019999    05    019  balance of county
3  059030999    05    030  balance of county
4  059034999    05    034  balance of county
5  059037999    05    037  balance of county
6  059043999    05    043  balance of county
County: Empty DataFrame
Columns: [ID, State, County, Area]
Index: []
1972_1982 Nom_Tax_Rate_Res_All_SFH_Med
BOC:           ID State County               Area
0  059001999    05    001  balance of county
1  059010999    05    010  balance of county
2  059019999    05    019  balance of county
3  059030999    05    030  balance of county
4  059034999    05    034  balance of county
5  059037999    05    037  balance of county
6  059043999    05    043  balance of county
County: Empty DataFrame
Columns: [ID, State, County, Area]
Index: []
1972_1982 Eff_Tax_Rate_All_Res_Med
BOC:           ID State

'No data at county level for 1972. No data at county level for 1967.'

In [38]:
COUNTY_CODE2COUNTY = dict(CA_data.loc[CA_data["Type"] == "County", ["County", "Area"]].groupby(["County", "Area"]).size().index)

# miss "038": San Francisco County
# 052038001- "San Francisco, CA" and "San Francisco City and County" are the same. It is a city, also a county.

COUNTY_CODE2COUNTY["038"] = "san francisco county, ca"

In [40]:
COUNTY_CODE2COUNTY

{'001': 'alameda county, ca',
 '007': 'contra costa county, ca',
 '010': 'fresno county, ca',
 '015': 'kern county, ca',
 '019': 'los angeles county, ca',
 '027': 'monterey county, ca',
 '030': 'orange county, ca',
 '033': 'riverside county, ca',
 '034': 'sacramento county, ca',
 '036': 'san bernardino county, ca',
 '037': 'san diego county, ca',
 '039': 'san joaquin county, ca',
 '041': 'san mateo county, ca',
 '042': 'santa barbara county, ca',
 '043': 'santa clara county, ca',
 '048': 'solano county, ca',
 '049': 'sonoma county, ca',
 '050': 'stanislaus county, ca',
 '056': 'ventura county, ca',
 '038': 'san francisco county, ca'}

In [39]:
print(cnt_tbl.loc[("1972_1982", "All"), "Balance of County"])
print(cnt_tbl.loc[("1977_1982", "All"), "Balance of County"])
new_boc_from1977_1982to1972_1982 = set(consistent_area_res["Balance of County"]["1972_1982"]["All"]) - set(consistent_area_res["Balance of County"]["1977_1982"]["All"])
print(new_boc_from1977_1982to1972_1982)


7
6
{'059030999'}


In [40]:
COUNTY_CODE2COUNTY['030']

'orange county, ca'

In [41]:
CA_data_na_cnt = CA_data.set_index(['ID', 'State', 'County', 'Type', 'Area', 'Year'])
assert CA_data.shape[0] == CA_data_na_cnt.shape[0]
CA_data_na_cnt = CA_data_na_cnt.isna().sum(axis=1).to_frame().reset_index()
pivot_CA_data_na_cnt = CA_data_na_cnt.pivot(index=['ID', 'State', 'County', 'Type', 'Area'], columns="Year")
pivot_CA_data_na_cnt = pivot_CA_data_na_cnt == 0
pivot_CA_data_na_cnt.columns = pivot_CA_data_na_cnt.columns.droplevel()

In [42]:
CA_data_na_cnt = CA_data_na_cnt.rename(columns={0: "have_all_indicators"})
CA_data_na_cnt["have_all_indicators"] = CA_data_na_cnt["have_all_indicators"] == 0
CA_data_na_cnt = CA_data_na_cnt.groupby(["Type", "Year"]).agg({"have_all_indicators": sum})

In [43]:
na_by_geo_and_year = CA_data_na_cnt.merge(na_by_geo_and_year, how="left", left_index=True, right_index=True)

In [44]:
na_by_geo_and_year

have_all_indicators  Nom_Tax_Rate_All_Res_Med  \
Type              Year                                                  
Balance of County 1967                    0                         0   
                  1972                   19                        19   
                  1977                   17                        17   
                  1982                    7                         7   
City              1967                    0                         0   
                  1972                   61                        61   
                  1977                   58                        58   
                  1982                   31                        34   
County            1967                    0                         0   
                  1972                    0                         0   
                  1977                   18                        18   
                  1982                    7                         7   

                        Nom_Tax_Rate_Res_All_SFH_Med  \
Type              Year                                 
Balance of County 1967                             7   
                  1972                            19   
                  1977                            17   
                  1982                             7   
City              1967                            14   
                  1972                            61   
                  1977                            58   
                  1982                            31   
County            1967                             0   
                  1972                             0   
                  1977                            18   
                  1982                             7   

                        Eff_Tax_Rate_All_Res_Med  \
Type              Year                             
Balance of County 1967                         0   
                  1972                        19   
                  1977                        17   
                  1982                         7   
City              1967                         0   
                  1972                        61   
                  1977                        58   
                  1982                        34   
County            1967                         0   
                  1972                         0   
                  1977                        18   
                  1982                         7   

                        Eff_Tax_Rate_Res_All_SFH_Med  AV_Ratio_All_Res_Med  
Type              Year                                                      
Balance of County 1967                             7                     0  
                  1972                            19                    19  
                  1977                            17                    17  
                  1982                             7                     7  
City              1967                            14                     0  
                  1972                            61                    61  
                  1977                            58                    58  
                  1982                            31                    34  
County            1967                             0                     0  
                  1972                             0                     0  
                  1977                            18                    18  
                  1982                             7                     7

# Investigate why BOC is not aligned with City in 1977

BOC = County - major cities(1967, 1982 for sure, 72,77 cannot find info)

## 1967
**Data are limited to a single property-use category (single-family houses) and
for much fewer areas compared to other years.**
In 1967, only has data for BOC and City for indicators: Nom_Tax_Rate_Res_All_SFH_Med and Eff_Tax_Rate_Res_All_SFH_Med.

**The areas listed consist of cities having 100,000 or more population in 1960, and also the remaining portion of each of counties in which one or more of the large cities is located. Cities and BOCs are not shown because of insufficient sales representation in the sample or multiple tax billing agencies.


## 1972
? what is city-county consolidation


## 1977

**Each of the cities of 50,000 population or more for which a sufficient sample of measurable sales was available. Wherever possible, the Bureau also calculated the effective rate applicable in the "balance of county" area involved, if any.**

CA: All assessed values and property tax bills used to compute ratios and tax rates for San Francisco CA are as of March 1, 1975.

## 1982

**These statistics were sought, in general, only for those cities with 1978 populations of 200,000 or more and their surrounding counties, including for those counties only, each city with a 1978 population between 50,000 and 200,000.**

CA: Balance of county rates and ratios apply to BOC having 1978 population of 50,000 or more. 

In [45]:
pivot_CA_data_na_cnt = pivot_CA_data_na_cnt.reset_index()
pivot_CA_data_na_cnt.columns = pivot_CA_data_na_cnt.columns.rename("")
def get_govs_have_all_indicators_by(gov_type, year):
    """List name of govs of <gov_type> that have all 5 indicators in <year>"""
    _df = pivot_CA_data_na_cnt.loc[(pivot_CA_data_na_cnt['Type'] == gov_type) & pivot_CA_data_na_cnt[year]].reset_index()
    return _df

In [46]:
AVAILABLE_YEARS = [#"1967", "1972", no county data  
                   "1977", "1982"]
for year in AVAILABLE_YEARS:
    _boc = get_govs_have_all_indicators_by("Balance of County", year)
    _county = get_govs_have_all_indicators_by("County", year)
    if not check_if_boc_align_with_county(_boc, _county):
        print(year)

1977


In [47]:
consis_1977_boc = get_govs_have_all_indicators_by("Balance of County", "1977")
consis_1977_county = get_govs_have_all_indicators_by("County", "1977")

In [48]:
assert len(set(list(consis_1977_boc['County'])) - set(list(consis_1977_county['County']))) == 0 # All BOCs have their respective counties.
print(set(list(consis_1977_county['County'])) - set(list(consis_1977_boc['County'])))

{'056'}


In [49]:
print(f"""{COUNTY_CODE2COUNTY["056"]} doesn't have its BOC""")

ventura county, ca doesn't have its BOC


Once Proposition 13 passed, property assessments for the 1978-79 fiscal year were required to be “rolled back” to 1975-76 values, establishing the first base year values in California. Properties that have not sold or undergone new construction since February 1975 are said to have a 1975 base year value.

# output 18 cities with all valid tax rate indicators

In [50]:
city_tax_rates_77_82 = lookup_for_area_by_ID(consistent_area_res["City"]["1977_1982"]["All"])

In [51]:
city_tax_rates_77_82.head()

,ID,State,County,Area
0,052001001,05,001,alameda city
1,052001003,05,001,berkeley city
2,052001005,05,001,fremont city
3,052001006,05,001,hayward city
4,052001009,05,001,oakland city


In [52]:
city_tax_rates_77_82 = city_tax_rates_77_82.merge(
    CA_data.loc[(CA_data["Year"] == "1977") | (CA_data["Year"] == "1982"),
                ["ID", "Year"] + VAL_COLUMNS].pivot(index="ID", columns="Year"), how="left", on="ID")
city_tax_rates_77_82.describe()

/usr/local/opt/anaconda3/envs/prop13_analysis/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,"(Nom_Tax_Rate_All_Res_Med, 1977)","(Nom_Tax_Rate_All_Res_Med, 1982)","(Nom_Tax_Rate_Res_All_SFH_Med, 1977)","(Nom_Tax_Rate_Res_All_SFH_Med, 1982)","(Eff_Tax_Rate_All_Res_Med, 1977)","(Eff_Tax_Rate_All_Res_Med, 1982)","(Eff_Tax_Rate_Res_All_SFH_Med, 1977)","(Eff_Tax_Rate_Res_All_SFH_Med, 1982)","(AV_Ratio_All_Res_Med, 1977)","(AV_Ratio_All_Res_Med, 1982)"
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,12.596667,1.182222,12.776667,1.183333,1.932222,0.597222,1.908333,0.626667,17.922222,54.701111
std,2.625222,0.103501,2.741831,0.105105,0.384701,0.156762,0.398796,0.166522,1.387573,13.301008
min,9.100000,1.000000,9.100000,1.000000,1.350000,0.340000,1.310000,0.300000,13.700000,30.630000
25%,10.877500,1.120000,10.892500,1.115000,1.592500,0.475000,1.570000,0.535000,17.400000,45.342500
50%,12.505000,1.160000,12.495000,1.165000,1.910000,0.615000,1.870000,0.645000,18.300000,57.750000
75%,13.692500,1.190000,14.277500,1.190000,2.247500,0.720000,2.257500,0.737500,18.975000,62.885000
max,20.170000,1.420000,20.260000,1.430000,2.550000,0.820000,2.560000,0.910000,19.200000,73.600000


In [53]:
_cols = [name[0] + "_" + name[1] if type(name) is tuple else name for name in city_tax_rates_77_82.columns]
city_tax_rates_77_82.columns = _cols
city_tax_rates_77_82["County"] = city_tax_rates_77_82["County"].map(COUNTY_CODE2COUNTY)
city_tax_rates_77_82.to_csv("../../../data/interim/city_with_tax_rate_in_1977_and_1982.csv", index=False)

# output: tax rates by year for CA cities

In [54]:
CA_data_city = CA_data.loc[CA_data["Type"] == "City", :].drop(["State", "Type"], axis=1)
CA_data_city["County"] = CA_data_city["County"].map(COUNTY_CODE2COUNTY)

assert CA_data_city[CA_data_city["County"].isna()].empty

CA_data_city.head()

,ID,County,Area,Year,Nom_Tax_Rate_All_Res_Med,Nom_Tax_Rate_Res_All_SFH_Med,Eff_Tax_Rate_All_Res_Med,Eff_Tax_Rate_Res_All_SFH_Med,AV_Ratio_All_Res_Med
25,052001001,"alameda county, ca",alameda city,1982,1.19,1.19,0.75,0.75,62.87
26,052001003,"alameda county, ca",berkeley city,1982,1.18,1.18,0.61,0.52,51.83
27,052001005,"alameda county, ca",fremont city,1982,1.35,1.35,0.74,0.74,62.15
28,052001006,"alameda county, ca",hayward city,1982,1.31,1.31,0.71,0.71,71.07
29,052001009,"alameda county, ca",oakland city,1982,1.31,1.31,0.60,0.60,50.60


In [55]:
# remove redundent ", ca" in county
assert sum(~CA_data_city["County"].str.endswith(", ca")) == 0

CA_data_city["County"] = CA_data_city["County"].str.replace(r", ca$", "", regex=True)

assert sum(CA_data_city["County"].str.endswith(", ca")) == 0

In [56]:
CA_data_city.to_csv("../../../data/interim/tax_rates_av_ratio_CA_city_by_year.csv", index=False)